
# Results & Plots
Combine TensorFlow and PyTorch metrics, generate comparison tables and charts.


In [1]:

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RESULTS_DIR = Path("results")
TF_METRICS = RESULTS_DIR / "tf_metrics.csv"
PT_METRICS = RESULTS_DIR / "pytorch_metrics.csv"

tf_df = pd.read_csv(TF_METRICS) if TF_METRICS.exists() else pd.DataFrame()
pt_df = pd.read_csv(PT_METRICS) if PT_METRICS.exists() else pd.DataFrame()

combined = pd.concat([tf_df, pt_df], ignore_index=True)
combined_path = RESULTS_DIR / "model_comparison.csv"
combined.to_csv(combined_path, index=False)
print(f"Combined metrics saved -> {combined_path}")
combined


Combined metrics saved -> results/model_comparison.csv


,framework,model,accuracy,precision,recall,f1,auc,train_time_sec,train_time_min,checkpoint
0,tensorflow,mobilenet_v2,0.806408,0.790043,0.999325,0.882445,0.975809,585.249605,9.754160,/Users/pratyaksh/UTA/sem3/CV/plantvillage-stud...
1,pytorch,alexnet,0.986865,0.989398,0.992571,0.990982,0.998929,1907.578894,31.792982,/Users/pratyaksh/UTA/sem3/CV/plantvillage-stud...


## Plots

In [2]:

def plot_bar(df, metric, title, filename):
    plt.figure(figsize=(8, 5))
    x = np.arange(len(df))
    plt.bar(x, df[metric])
    plt.xticks(x, df["framework"] + ":" + df["model"], rotation=30, ha="right")
    plt.ylabel(metric)
    plt.title(title)
    plt.tight_layout()
    out_path = RESULTS_DIR / filename
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"Saved {out_path}")

if not combined.empty:
    sort_df = combined.sort_values("accuracy", ascending=False)
    plot_bar(sort_df, "accuracy", "Accuracy by model", "accuracy_by_model.png")
    plot_bar(sort_df, "f1", "F1-score by model", "f1_by_model.png")
    plot_bar(sort_df, "auc", "AUC by model", "auc_by_model.png")
    plot_bar(sort_df, "train_time_min", "Training time (min) by model", "train_time_by_model.png")


Saved results/accuracy_by_model.png
Saved results/f1_by_model.png
Saved results/auc_by_model.png
Saved results/train_time_by_model.png
